# 03 - DuckDB Queries
DuckDB is an in-process SQL OLAP database management system. [duckdb.org](http://duckdb.org)

With the timeseries, geospatial, crosswalk and attribute table data stored in parquet files, using DuckBD we can explore the data using SQL.

NOTE: The example code provided is not safe from SQL injection.  It is provided to demonstrate the power of DuckDB to query data stored in Parquet files. Also, this is not a web service.

In [1]:
import duckdb
from pathlib import Path

In [2]:
CACHE_DIR = Path(Path.home(), "shared", "rti-eval")
STUDY_DIR = Path(CACHE_DIR, "post-event-example")
USGS = Path(STUDY_DIR, "timeseries/usgs/*.parquet")
MEDIUM_RANGE_MEM1 = Path(STUDY_DIR, "timeseries/medium_range_mem1/*.parquet")
SHORT_RANGE = Path(STUDY_DIR, "timeseries/short_range/*.parquet")
CROSSWALK = Path(STUDY_DIR, "geo/usgs_nwm22_crosswalk.parquet")
GEOMETRY = Path(STUDY_DIR, "geo/usgs_geometry.parquet")

We will write a few simple SQL queries to demonstrate how DuckBD can be used to query the data we stored in the Parquet files.

First, lets just see how many rows are in one of the timeseries "tables".

In [3]:
duckdb.query(f"SELECT count(*) FROM read_parquet('{SHORT_RANGE}');")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    105601536 │
└──────────────┘

Now lets see how many unique `referece_times` are in the timeseries table.

In [4]:
duckdb.query(f"SELECT count(DISTINCT reference_time) as reference_time_count FROM read_parquet('{SHORT_RANGE}');")

┌──────────────────────┐
│ reference_time_count │
│        int64         │
├──────────────────────┤
│                  768 │
└──────────────────────┘

See a single row from from the timeseries data.

In [5]:
duckdb.query(f"""
    SELECT * FROM read_parquet('{SHORT_RANGE}') LIMIT 1;
""")

┌─────────────────────┬───────────────┬────────────────────┬───┬───────────────┬───────────────┬───────────────────┐
│     value_time      │  location_id  │       value        │ … │ configuration │ variable_name │ __index_level_0__ │
│      timestamp      │    varchar    │       double       │   │    varchar    │    varchar    │       int64       │
├─────────────────────┼───────────────┼────────────────────┼───┼───────────────┼───────────────┼───────────────────┤
│ 2023-01-02 17:00:00 │ nwm22-7086109 │ 0.2199999950826168 │ … │ short_range   │ streamflow    │                 0 │
├─────────────────────┴───────────────┴────────────────────┴───┴───────────────┴───────────────┴───────────────────┤
│ 1 rows                                                                                       8 columns (6 shown) │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

Now, just to show the power, lets join the observed USGS data to the short range forecast data and select a single forecast.

In [6]:
duckdb.query(f"""
    SELECT 
        u.location_id as primary_location_id,
        sr.location_id as secondary_location_id,
        sr.reference_time as reference_time,
        u.value_time as value_time,
        u.value as primary_value,
        sr.value as secondary_value,
    FROM read_parquet('{SHORT_RANGE}') sr
    JOIN read_parquet('{CROSSWALK}') cw 
        ON sr.location_id = cw.secondary_location_id 
    JOIN read_parquet('{USGS}') u 
        ON cw.primary_location_id = u.location_id
        AND sr.value_time = u.value_time
    WHERE
        primary_location_id = 'usgs-10336676' AND
        sr.reference_time = '2023-01-02 16:00:00' AND
        primary_value > 0
    ORDER BY 
        sr.value_time DESC
    LIMIT 10;
""").to_df()

,primary_location_id,secondary_location_id,reference_time,value_time,primary_value,secondary_value
0,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 10:00:00,1.087367,0.04
1,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 09:00:00,1.121347,0.04
2,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 08:00:00,1.121347,0.05
3,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 07:00:00,1.155327,0.06
4,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 06:00:00,1.155327,0.08
5,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 05:00:00,1.189308,0.10
6,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 04:00:00,1.189308,0.13
7,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 03:00:00,1.189308,0.17
8,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 02:00:00,1.226119,0.22
9,usgs-10336676,nwm22-8941685,2023-01-02 16:00:00,2023-01-03 01:00:00,1.226119,0.29


This starts to get trciky as the queries get more complex.  This is where TEEHR queries library comes in.